In [ ]:
%matplotlib inline
from IPython.display import clear_output
from pytesseract import image_to_string

from srbot import *

In [ ]:
#Draynor willow chopper banker

bank_icon = load_image('bank_icon.png')

mm_willow = [24,30,1]
mm_water = [125,144,185]

target()
click_mouse(mmxc,mmyc)
done = False
inventories = 0
last_log = mark_time()
while True:
    client = get_client()
    if len(find_bitmap(loginscreen,client)) > 0:
        login()
        continue
    if mark_time() - last_log > 10*60:
        raise RuntimeError('Been a loooong time since we got wood. Probably dead.')
        
    set_compass_angle(90,tol=7,click=False)
    inv = count_inv()
    if inv >= 28: #go bank
        minimap = get_minimap()
        bank = find_best_bitmap(bank_icon,minimap,mode='xcorr',tol=0.15)
        print('bank icon:',len(bank))
        if len(bank) > 0:
            click_mouse(*(bank[np.random.randint(len(bank))]+[mmxs+5,mmys+5]))
            flag_wait()
            sleep(1.0)
            mainscreen = get_mainscreen()
            a = find_colors([146,141,140],mainscreen,tol=0.05,mode='hsl')
            b = find_colors([166,149,115],mainscreen,tol=0.05,mode='hsl')
            points = filter_near(b,a,15)
            np.random.shuffle(points)
            if len(points) > 1:
                minidx = np.argmin(np.sum(np.square(points-[msxc-msxs,msyc-msys]),axis=1))
                points[-1],points[minidx] = points[minidx],points[-1]
            for point in points[-5:]:
                click_mouse(*point,left=False)
                sleep(0.05)
                use = find_bitmap(use_booth,get_client())
                if len(use) > 0:
                    click_mouse(*(use[0]+[10,10]))
                    flag_wait()
                    sleep(1.0)
                    deposit_all()
                    sleep(0.5)
                    click_mouse(488,43)
                    clear_output()
                    inventories += 1
                    print('finished %i inventories'%inventories)
                    sleep(0.5)
                    break
                move_mouse(*(point+[0,-25]))      
    else: #go chop
        mainscreen = get_mainscreen()
        a = find_colors([120,96,73],mainscreen,tol=0.05,mode='hsl')
        b = find_colors([136,160,42],mainscreen,tol=0.05,mode='hsl')
        trees = filter_near(a,b,20)
        chopped = False
        if len(trees) > 0:
            clusters,counts = cluster(trees)
            trees = np.asarray([np.mean(cluster.T,axis=1) for cluster in clusters])
            trees = trees[np.argsort(np.sum(np.square(trees-[msxc-msxs,msyc-msys]),axis=1))]
            for tree in trees[:min(10,len(trees))]:
                move_mouse(*(tree+[msxs,msys]))
                sleep(0.2)
                uptext = get_uptext()
                cyan = find_colors([0,238,238],uptext,tol=0.2)
                print('tree cyan',len(cyan))
                if len(cyan) > 130 and len(cyan) < 180: #it's a willow!
                    chopped = True
                    click_mouse(*(tree+[msxs,msys]))
                    flag_wait()
                    i = 0
                    inv = count_inv()
                    while True:
                        i = i+1
                        if inv != count_inv():
                            inv = count_inv()
                            last_log = mark_time()
                            i = 0
                        if inv >= 28:
                            print('inventory full')
                            break
                        if i > 10:
                            print('chopping timed out')
                            break
                        uptext = get_uptext()
                        cyan = find_colors([0,238,238],uptext,tol=0.2)
                        if len(cyan) < 130 or len(cyan) > 180:
                            print('tree disappeared')
                            break
                        sleep(1.0+np.random.random()*1.0)
                    break
        if chopped:
            continue
        print('trying new area')
        if np.random.random() < 0.2:
            run_on()
        minimap = get_minimap()
        a = find_colors(mm_willow,minimap,tol=0.12,mode='dist')
        b = find_colors(mm_water,minimap,tol=(0.05,0.2,0.1),mode='hsl')
        willows = filter_near(a,b,20)
        willows = filter_radius(willows,[mmxc-mmxs,mmyc-mmys],55)
        clusters,counts = cluster(willows,radius=5)
        mask = np.logical_and(counts>5,counts<20)
        if np.count_nonzero(mask) > 0:
            willows = np.concatenate(clusters[mask])
            clusters,counts = cluster(willows,radius=20)
            willows = clusters[np.argmax(counts)]
            willow = willows[np.random.randint(len(willows))]
            click_mouse(*(willow+[mmxs,mmys]))
            flag_wait()
        else: 
            print('hopelessly lost')
        